In [1]:
!pip install papermill
import papermill as pm
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
import os
import uuid
import tempfile
import pathlib

  Using cached papermill-2.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached entrypoints-0.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached ansicolors-1.1.8-py2.py3-none-any.whl.metadata (9.0 kB)
Using cached papermill-2.6.0-py3-none-any.whl (38 kB)
Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
Using cached ansicolors-1.1.8-py2.py3-none-any.whl (13 kB)
Using cached entrypoints-0.4-py3-none-any.whl (5.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [papermill]


# Serial Execution

In [2]:
notebook_in  = "Main_Modified_Update.ipynb"      # the master notebook
notebook_out = "output_notebook"

cases = [
    ("ksplit1",  True),
    ("ksplit2",  True),
    ("ksplit3",  True),
    ("Control", False),
    ("std",      True),
    ("kblock1",  True),
    ("kblock2",  True),
    ("kblock3",  True)
    
]

In [3]:
total_start = time.perf_counter()
for label, val_sim in cases:
    print(f"Running {label} …")
    t0 = time.perf_counter()
    
    pm.execute_notebook(
        notebook_in,
        notebook_out,
        parameters=dict(
            validate_type = label,
            validation_sim = val_sim
        ),
        report_mode=False      # keep full executed notebook
    )

    elapsed = time.perf_counter() - t0
    print(f"{label} finished in {elapsed:6.1f} s")

total_elapsed = time.perf_counter() - total_start
print(f"Done. {len(cases)} model runs executed in {total_elapsed:6.1f} s.")

Running ksplit1 …


Executing:   0%|          | 0/48 [00:00<?, ?cell/s]

/srv/conda/envs/notebook/lib/python3.12/site-packages/papermill/iorw.py:149: UserWarning: the file is not specified with any extension : output_notebook
  warnings.warn(f"the file is not specified with any extension : {os.path.basename(path)}")


KeyboardInterrupt: 

# Parallel Execution

In [4]:
def run_case(label_valsim):
    """Execute one scenario in a separate process and discard the output nb."""
    label, val_sim = label_valsim
    t0 = time.perf_counter()

    # unique temp file (same extension as notebook_out but isolated per run)
    tmp_out = pathlib.Path(tempfile.gettempdir()) / f"{uuid.uuid4().hex}.ipynb"

    try:
        pm.execute_notebook(
            notebook_in,
            str(tmp_out),                    # write here, delete later
            parameters=dict(
                validate_type  = label,
                validation_sim = val_sim
            ),
            report_mode=False,
            log_output=False,
            progress_bar=False
        )
    finally:
        # best-effort cleanup
        try:
            tmp_out.unlink(missing_ok=True)
        except Exception:
            pass

    elapsed = time.perf_counter() - t0
    return label, elapsed

In [7]:
# --- original workflow values -----------------
notebook_in  = "Main_Modified_Update.ipynb"   # template
notebook_out = "output_notebook"              # kept only for consistency
cases = [
    ("ksplit1",  True),
    ("ksplit2",  True),
    ("ksplit3",  True),
    ("Control",  False),
    ("std",      True),
    ("kblock1",  True),
    ("kblock2",  True),
    ("kblock3",  True)
]
max_workers = min(2, os.cpu_count() or 1)     # parallelism cap

In [8]:
if __name__ == "__main__":
    print(f"Launching {len(cases)} runs on up to {max_workers} workers …")
    total_start = time.perf_counter()

    with ProcessPoolExecutor(max_workers=max_workers) as pool:
        futures = {pool.submit(run_case, c): c[0] for c in cases}

        for fut in as_completed(futures):
            label, secs = fut.result()
            print(f"{label:8} finished in {secs:6.1f} s")

    total_elapsed = time.perf_counter() - total_start
    print(f"\n All {len(cases)} runs complete in {total_elapsed:6.1f} s")

Launching 8 runs on up to 2 workers …
ksplit1  finished in 2326.6 s
ksplit2  finished in 2498.4 s
ksplit3  finished in 2525.8 s
Control  finished in 3131.0 s
kblock1  finished in 4501.0 s
std      finished in 7810.0 s
kblock2  finished in 5130.3 s
kblock3  finished in 5348.9 s

 All 8 runs complete in 18011.4 s
